In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

import pandas as pd


dosya_yolu = '/content/drive/MyDrive/hackaton/hackathon_train_set.csv'


df = pd.read_csv(dosya_yolu, sep=';')

print(df.head())

print(df.info())

       District                  Neighborhood        Price Adrtisement Date  \
0      Güngören  Abdurrahman Nafiz Gürman Mh.   620.000 TL       23.12.2019   
1      Bağcılar                  100. Yıl Mh.   115.000 TL       04.01.2020   
2      Ataşehir                     İnönü Mh.   419.000 TL       06.01.2020   
3  Büyükçekmece               Cumhuriyet Mah.   395.000 TL       26.12.2019   
4      Çekmeköy               Cumhuriyet Mah.   485.000 TL       30.12.2019   

   m² (Gross)  m² (Net) Number of rooms   Building Age Floor location  \
0         150     140.0             3+1  26-30 between              3   
1          85      80.0             1+1  16-20 between   Garden Floor   
2         120     100.0             3+1              1              5   
3         143     124.0             3+1              3   Garden Floor   
4         139     115.0             3+1   5-10 between              8   

  Number of floors  ... Top floor Garage / Shop Top Entrance floor  \
0               

In [ ]:
# Düzeltilmiş veri setinin ilk 5 satırını görüntülüyoruz.
print(df.head())

       District                  Neighborhood        Price Adrtisement Date  \
0      Güngören  Abdurrahman Nafiz Gürman Mh.   620.000 TL       23.12.2019   
1      Bağcılar                  100. Yıl Mh.   115.000 TL       04.01.2020   
2      Ataşehir                     İnönü Mh.   419.000 TL       06.01.2020   
3  Büyükçekmece               Cumhuriyet Mah.   395.000 TL       26.12.2019   
4      Çekmeköy               Cumhuriyet Mah.   485.000 TL       30.12.2019   

   m² (Gross)  m² (Net) Number of rooms   Building Age Floor location  \
0         150     140.0             3+1  26-30 between              3   
1          85      80.0             1+1  16-20 between   Garden Floor   
2         120     100.0             3+1              1              5   
3         143     124.0             3+1              3   Garden Floor   
4         139     115.0             3+1   5-10 between              8   

  Number of floors  ... Top floor Garage / Shop Top Entrance floor  \
0               

In [ ]:
column_headers = df.columns.tolist()
for col in column_headers:
    print(col)

District
Neighborhood
Price
Adrtisement Date
m² (Gross)
m² (Net)
Number of rooms
Building Age
Floor location
Number of floors
Heating
Number of bathrooms
Balcony
Furnished
Using status
Available for Loan
From who
Swap
Front West
Front East
Front South
Front North
ADSL
Wood Joinery
Smart House
Alarm (Thief)
Alarm (Fire)
Alaturka Toilet
Aluminum joinery
American Door
American kitchen
Built-in on
Elevator
Balcony.1
Barbecue
Household appliances
Painted
Dishwasher
Refrigerator
Wall paper
Shower
Parents' Bathroom
Fiber Internet
On
Dressing room
Closet
Video intercom
Hilton Bathroom
Intercom System
Insulating glass
Jacuzzi
Plasterboard
Cellar
Air conditioning
Tub
Laminate Floor
Marley
Furniture
Kitchen (Built-in)
Kitchen (Laminate)
Kitchen Natural Gas
PVC Joinery
Blinds
Parquet Floor
Ceramic Floor
Set Top Cooker
Spot Lighting
Terrace
Thermosiphon
Cloakroom
Wi-Fi
Face Recognition & Fingerprint
Washing Machine
Washing machine
Laundry room
Steel door
Water heater
Fireplace
Elevator.1
Steam room

In [ ]:
# Sütun isimlerini düzeltiyoruz
df.rename(columns={
    'Throat': 'Bosphorus_View',       # En kritik düzeltme
    'Scaffolding': 'Ferry_Dock',      # Ulaşım için önemli
    'Filled': 'Shared_Taxi_Dolmus',   # Ulaşım
    'Unirsity': 'University',         # Yazım hatası
    'Rerse Duplex': 'Reverse_Duplex', # Yazım hatası
    'The mall': 'Shopping_Mall',      # Daha standart isimlendirme
    'Next to sea shore': 'Seafront'   # Daha profesyonel terim
}, inplace=True)

In [ ]:

# Sütunlar 1 ve 0'lardan (veya True/False) oluşuyorsa:
df['Has_Pool'] = (
    df['Swimming Pool (Open)'] |
    df['Swimming Pool (Indoor)'] |
    df['With Private Pool'] |
    df['Pool'] |
    df['Swimming pool']
).astype(int)

# 2. İsterseniz eski dağınık sütunları silebilirsiniz
cols_to_drop = ['Swimming Pool (Open)', 'Swimming Pool (Indoor)', 'With Private Pool', 'Pool', 'Swimming pool']
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# Genel bir "Otopark Var mı?" sütunu
df['Has_Parking'] = (
    df['Closed Garage'] |
    df['Car park'] |
    df['Parking Lot'] |
    df['Garage / Shop Top']
).astype(int)

# Detay sütunu: Kapalı otopark ayrıca değerlidir, onu ayrı bir sütun olarak tutabiliriz (rename ederek)
df.rename(columns={'Closed Garage': 'Indoors_Parking'}, inplace=True)

# Gereksizleri atalım
df.drop(columns=['Car park', 'Parking Lot', 'Garage / Shop Top'], inplace=True)

In [ ]:
# Bu özelliklerden en az 2 tanesi varsa o ev "Engelli Dostu"dur diyebiliriz.
accessibility_cols = ['Wide Corridor', 'Entrance / Ramp', 'Handle / Railing']

# Toplam kaç özellik var?
df['Accessibility_Score'] = df[accessibility_cols].sum(axis=1)

# Veya basitçe "Erişilebilir mi?"
df['Is_Handicap_Accessible'] = (df['Accessibility_Score'] > 0).astype(int)

# Eski sütunları temizleyelim
df.drop(columns=accessibility_cols, inplace=True)

In [ ]:
# Park ve Park.1 muhtemelen aynıdır. Birleştiriyoruz.
df['Near_Park'] = (df['Park'] | df['Park.1']).astype(int)
df.drop(columns=['Park', 'Park.1'], inplace=True)

# Asansör: Elevator.2 muhtemelen yük asansörüdür ama emin değilsek ve 'Elevator' diye başka sütun yoksa ismini düzeltelim.
# Eğer 'Elevator' diye başka sütun varsa birleştirelim:
if 'Elevator' in df.columns:
    df['Has_Elevator'] = (df['Elevator'] | df['Elevator.2']).astype(int)
    df.drop(columns=['Elevator', 'Elevator.2'], inplace=True)
else:
    df.rename(columns={'Elevator.2': 'Has_Elevator'}, inplace=True)

In [ ]:
# Düzeltilmiş veri setinin ilk 5 satırını görüntülüyoruz.
print(df.head())

       District                  Neighborhood        Price Adrtisement Date  \
0      Güngören  Abdurrahman Nafiz Gürman Mh.   620.000 TL       23.12.2019   
1      Bağcılar                  100. Yıl Mh.   115.000 TL       04.01.2020   
2      Ataşehir                     İnönü Mh.   419.000 TL       06.01.2020   
3  Büyükçekmece               Cumhuriyet Mah.   395.000 TL       26.12.2019   
4      Çekmeköy               Cumhuriyet Mah.   485.000 TL       30.12.2019   

   m² (Gross)  m² (Net) Number of rooms   Building Age Floor location  \
0         150     140.0             3+1  26-30 between              3   
1          85      80.0             1+1  16-20 between   Garden Floor   
2         120     100.0             3+1              1              5   
3         143     124.0             3+1              3   Garden Floor   
4         139     115.0             3+1   5-10 between              8   

  Number of floors  ... Triplex Ground floor Roof Duplex Pick Up Data Time  \
0       

In [ ]:
cd '*/drive/MyDrive/hackathon/proje_analiz_sonucu.csv'

[Errno 2] No such file or directory: '*/drive/MyDrive/hackathon/proje_analiz_sonucu.csv'
/content


In [ ]:
ls

drive/  sample_data/


In [ ]:
# Veriyi kaydetmek istediğimiz yeni dosya yolunu ve ismini belirliyoruz.
# Orijinal dosyanın üzerine yazmamak için ismini değiştirmek (örn: _sonuc veya _v2 eklemek) iyi bir pratiktir.
# Buradaki klasör yolunu kendi Drive yapına göre güncellemeyi unutma.
kayit_yolu = 'proje_analiz_sonucu.csv'

# DataFrame'imizi (df) CSV formatına çevirip belirttiğimiz yola kaydediyoruz.
# index=False parametresi ÇOK ÖNEMLİDİR:
# Eğer bunu yazmazsak, Pandas satır numaralarını (0, 1, 2...) en başa yeni bir sütun olarak ekler.
# Genellikle temiz bir dosyada bu indeks numaralarını istemeyiz.
df.to_csv(kayit_yolu, index=False)

# İşlemin bittiğini anlamak için ekrana bir bilgi mesajı yazdırıyoruz.
print(f"Dosya başarıyla şu konuma kaydedildi: {kayit_yolu}")

Dosya başarıyla şu konuma kaydedildi: proje_analiz_sonucu.csv


In [ ]:
import pandas as pd
import numpy as np

def veri_setini_iyilestir(df):
    # 1. TEMEL TEMİZLİK (Fiyat ve Metrekareyi Sayıya Çevirme)
    # -------------------------------------------------------
    df['Price'] = df['Price'].astype(str).str.replace('.', '').str.replace(' TL', '')
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

    # Metrekare temizliği
    df['m² (Gross)'] = pd.to_numeric(df['m² (Gross)'], errors='coerce')
    df['m² (Net)'] = pd.to_numeric(df['m² (Net)'], errors='coerce')

    # Kat Sayısı Temizliği
    df['Number of floors'] = pd.to_numeric(df['Number of floors'], errors='coerce')

    # Fiyatı veya metrekaresi boş (NaN) olanları direkt sil, kurtarılmaz.
    df = df.dropna(subset=['Price', 'm² (Gross)', 'Number of floors'])

    # 2. MANTIKSAL DÜZELTME (Deniz Manzarası Yalanını Yakala)
    # -------------------------------------------------------
    # Deniz kenarı olmayan ilçelerin listesi (Örnek)
    ic_ilceler = ['Bağcılar', 'Esenler', 'Güngören', 'Sultanbeyli', 'Sancaktepe', 'Ümraniye', 'Kağıthane']

    # Bu ilçelerde Sea veya Throat 1 ise, zorla 0 yapıyoruz.
    maske = df['District'].isin(ic_ilceler)
    df.loc[maske, 'Sea'] = 0
    df.loc[maske, 'Throat'] = 0
    print(f"Mantıksal Düzeltme: İç ilçelerdeki hatalı manzaralar silindi.")

    # 3. FİZİKSEL DÜZELTME (Net > Brüt Olamaz)
    # -------------------------------------------------------
    # Net m2, Brüt m2'den büyükse, muhtemelen kullanıcı ters girmiştir. Yerlerini değiştir.
    hatali_m2 = df['m² (Net)'] > df['m² (Gross)']

    # Yer değiştirme işlemi (Swap)
    df.loc[hatali_m2, ['m² (Gross)', 'm² (Net)']] = df.loc[hatali_m2, ['m² (Net)', 'm² (Gross)']].values
    print(f"Fiziksel Düzeltme: {hatali_m2.sum()} adet hatalı metrekare girişi düzeltildi.")

    # 4. İSTATİSTİKSEL KESİM (Aşırı Uçuk Fiyatları At)
    # -------------------------------------------------------
    # Fiyatı 50.000 TL altı (Hatalı giriş) veya 50 Milyon TL üstü (Ultra lüks/Hata) olanları atalım.
    # Bu limitleri projene göre esnetebilirsin.
    df = df[(df['Price'] > 50000) & (df['Price'] < 50000000)]

    # Metrekaresi 10 m2 altı veya 1000 m2 üstü olanları at (Ofis veya saray olabilir)
    df = df[(df['m² (Gross)'] > 10) & (df['m² (Gross)'] < 1000)]

    print(f"Sonuç: Veri seti temizlendi. Yeni satır sayısı: {len(df)}")
    return df

# Fonksiyonu çalıştır
clean_df = veri_setini_iyilestir(df)

Mantıksal Düzeltme: İç ilçelerdeki hatalı manzaralar silindi.
Fiziksel Düzeltme: 2 adet hatalı metrekare girişi düzeltildi.
Sonuç: Veri seti temizlendi. Yeni satır sayısı: 27377


/tmp/ipython-input-2429867979.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[maske, 'Throat'] = 0


In [ ]:
# Veriyi kaydetmek istediğimiz yeni dosya yolunu ve ismini belirliyoruz.
# Orijinal dosyanın üzerine yazmamak için ismini değiştirmek (örn: _sonuc veya _v2 eklemek) iyi bir pratiktir.
# Buradaki klasör yolunu kendi Drive yapına göre güncellemeyi unutma.
kayit_yolu = 'clean_data_1. csv'

# DataFrame'imizi (df) CSV formatına çevirip belirttiğimiz yola kaydediyoruz.
# index=False parametresi ÇOK ÖNEMLİDİR:
# Eğer bunu yazmazsak, Pandas satır numaralarını (0, 1, 2...) en başa yeni bir sütun olarak ekler.
# Genellikle temiz bir dosyada bu indeks numaralarını istemeyiz.
df.to_csv(kayit_yolu, index=False)

# İşlemin bittiğini anlamak için ekrana bir bilgi mesajı yazdırıyoruz.
print(f"Dosya başarıyla şu konuma kaydedildi: {kayit_yolu}")

Dosya başarıyla şu konuma kaydedildi: clean_data_1.csv
